In [1]:
from pathlib import Path
import sys

import pandas as pd
import numpy as np

sys.path.append('../')
import payulator as pl


%load_ext autoreload
%autoreload 2

In [2]:
# Test some

principal = 11000
interest = 0.099
compounding_freq = 'monthly'
payment_freq = 'monthly'
num_years = 3
fee = 350
start_date = '2018-03-06'

A = pl.amortize(principal, interest, compounding_freq, payment_freq, num_years, 
  fee, start_date)

A


{'interest_and_fee_total': 2109.22,
 'interest_total': 1759.22,
 'num_payments': 36,
 'payment_schedule':     payment_sequence payment_date  beginning_balance  principal_payment  \
 0                  1   2018-03-06           11000.00             263.67   
 1                  2   2018-04-06           10736.33             265.85   
 2                  3   2018-05-06           10470.48             268.04   
 3                  4   2018-06-06           10202.44             270.25   
 4                  5   2018-07-06            9932.18             272.48   
 5                  6   2018-08-06            9659.70             274.73   
 6                  7   2018-09-06            9384.97             277.00   
 7                  8   2018-10-06            9107.98             279.28   
 8                  9   2018-11-06            8828.69             281.59   
 9                 10   2018-12-06            8547.11             283.91   
 10                11   2019-01-06            8263.20      

In [3]:
# path = "~/lending/loans/jack_haldane-willis/payment_schedule.csv"
# A['payment_schedule'].to_csv(path, index=False)

In [8]:
principal = 11000
interest = 0.099
compounding_freq = 'monthly'
payment_freq = 'monthly'
num_years = 2
fee = 350
start_date = '2018-03-06'

A = pl.amortize(principal, interest, compounding_freq, payment_freq, num_years, 
  fee, start_date)
B = pl.amortize(principal, interest, compounding_freq, 'weekly', num_years, 
  fee, '2019-03-07')

g = pl.aggregate_payment_schedules([A['payment_schedule'], B['payment_schedule']])
g

,principal_payment,interest_payment
payment_sequence,,
1,512.11,111.63
2,515.73,108.01
3,519.37,104.36
4,523.05,100.69
5,526.75,96.99
6,530.49,93.25
7,534.25,89.49
8,538.03,85.70
9,541.86,81.88


In [ ]:
# Someday/maybe create a Loan class and include ``loan_code`` as an attribute.
# Then log the contributing loan codes in a new column of ``aggregate_payment_schedules``